In [1]:
%cd ../..

/Users/kedarwaghmode/projects/personal/Modern-Time-Series-Forecasting-with-Python


In [2]:
import os
import time
import shutil

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import joblib
pio.templates.default = "plotly_white"

import warnings
from pathlib import Path

import humanize
from darts.metrics import mae, mase, mse
from sklearn.preprocessing import StandardScaler
from src.forecasting.ml_forecasting import (
    FeatureConfig,
    MissingValueConfig,
    MLForecast,
    ModelConfig,
    calculate_metrics,
)
from src.utils import plotting_utils
from src.utils.general import LogTime
from src.utils.ts_utils import darts_metrics_adapter, forecast_bias
from tqdm.autonotebook import tqdm
from src.forecasting.ml_forecasting import calculate_metrics
from src.utils import ts_utils
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

/Users/kedarwaghmode/projects/personal/Modern-Time-Series-Forecasting-with-Python/src/utils/data_utils.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
os.makedirs("imgs/chapter_12", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")
output = Path("data/london_smart_meters/output")

## Utility Functions

In [4]:
def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        )
    )
    return fig

In [5]:
def evaluate_forecast(y_pred, test_target, train_target, model_name):
    metric_l = []
    for _id in tqdm(test_target.index.get_level_values(0).remove_unused_categories().categories, desc="Calculating metrics..."):
        target = test_target.xs(_id)
        _y_pred = y_pred.xs(_id)
        history = train_target.xs(_id)
        metric_l.append(
            calculate_metrics(target, _y_pred, name=model_name, y_train=history)
        )
    eval_metrics_df = pd.DataFrame(metric_l)
    agg_metrics = {
            "Algorithm": model_name,
            "MAE": ts_utils.mae(
                test_target, y_pred
            ),
            "MSE": ts_utils.mse(
                test_target, y_pred
            ),
            "meanMASE": eval_metrics_df.loc[:, "MASE"].mean(),
            "Forecast Bias": ts_utils.forecast_bias_aggregate(
                test_target, y_pred
            )
    }
    return agg_metrics, eval_metrics_df

In [6]:
from itertools import cycle


def plot_forecast(pred_df, forecast_columns, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)
    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [
        "rgba(" + ",".join([str(c) for c in plotting_utils.hex_to_rgb(c)]) + ",<alpha>)"
        for c in px.colors.qualitative.Plotly
    ]
    act_color = colors[0]
    colors = cycle(colors[1:])
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].energy_consumption,
            mode="lines",
            line=dict(color=act_color.replace("<alpha>", "0.9")),
            name="Actual Consumption",
        )
    )
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines",
                line=dict(dash="dot", color=next(colors).replace("<alpha>", "1")),
                name=display_col,
            )
        )
    return fig

def highlight_abs_min(s, props=''):
    return np.where(s == np.nanmin(np.abs(s.values)), props, '')

## Reading the data

In [7]:
try:
    #Reading the missing value imputed and train test split data
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    # Read in the Validation dataset as test_df so that we predict on it
    test_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")
    # test_df = pd.read_parquet(preprocessed/"block_0-7_test_missing_imputed_feature_engg.parquet")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

Selecting only half the blocks to make deep learning training smoother, faster and fit in GPU memory. This is purely a hardware constraint and done beacause dealing with the engineering challenges of handling huge datasets is out of scope for this book.

In [8]:
uniq_blocks = train_df.file.unique().tolist()
sel_blocks = sorted(uniq_blocks, key=lambda x: int(x.replace("block_","")))[:len(uniq_blocks)//2]
train_df = train_df.loc[train_df.file.isin(sel_blocks)]
test_df = test_df.loc[test_df.file.isin(sel_blocks)]
sel_lclids = train_df.LCLid.unique().tolist()

In [9]:
target = "energy_consumption"
index_cols = ["LCLid", "timestamp"]

In [10]:
# Setting the indices
train_df.set_index(index_cols, inplace=True, drop=False)
test_df.set_index(index_cols, inplace=True, drop=False)
# Creating a pred_df with actuals
pred_df = pd.concat([train_df[[target]], test_df[[target]]])

In [11]:
global_ml_fc_df = pd.read_pickle(output/"gfm_predictions_val_df.pkl")
global_ml_fc_df.head()

global_ml_fc_df = global_ml_fc_df.loc[global_ml_fc_df.index.get_level_values(0).isin(sel_lclids)]
global_ml_fc_df

energy_consumption  GFM Baseline  \
LCLid     timestamp                                               
MAC000066 2014-01-01 00:00:00               0.311      0.163842   
          2014-01-01 00:30:00               0.170      0.204013   
          2014-01-01 01:00:00               0.158      0.157482   
          2014-01-01 01:30:00               0.178      0.157622   
          2014-01-01 02:00:00               0.142      0.159654   
...                                           ...           ...   
MAC005521 2014-01-31 21:30:00               0.114      0.144956   
          2014-01-31 22:00:00               0.081      0.142345   
          2014-01-31 22:30:00               0.120      0.106617   
          2014-01-31 23:00:00               0.321      0.134236   
          2014-01-31 23:30:00               0.080      0.288657   

                               GFM+Meta  (NativeLGBM)  Tuned GFM+Meta  \
LCLid     timestamp                                                     
MAC000066 2014-01-01 00:00:00                0.161060        0.178426   
          2014-01-01 00:30:00                0.201842        0.187700   
          2014-01-01 01:00:00                0.154096        0.155929   
          2014-01-01 01:30:00                0.153294        0.157074   
          2014-01-01 02:00:00                0.161306        0.157256   
...                                               ...             ...   
MAC005521 2014-01-31 21:30:00                0.146877        0.117701   
          2014-01-31 22:00:00                0.143741        0.108651   
          2014-01-31 22:30:00                0.107959        0.099839   
          2014-01-31 23:00:00                0.142867        0.112869   
          2014-01-31 23:30:00                0.289984        0.215584   

                               Tuned GFM+Meta+Random Part  \
LCLid     timestamp                                         
MAC000066 2014-01-01 00:00:00                    0.174797   
          2014-01-01 00:30:00                    0.200103   
          2014-01-01 01:00:00                    0.151358   
          2014-01-01 01:30:00                    0.160108   
          2014-01-01 02:00:00                    0.159512   
...                                                   ...   
MAC005521 2014-01-31 21:30:00                    0.117419   
          2014-01-31 22:00:00                    0.117147   
          2014-01-31 22:30:00                    0.105532   
          2014-01-31 23:00:00                    0.114253   
          2014-01-31 23:30:00                    0.200526   

                               Tuned GFM+Meta+ACORN Part  \
LCLid     timestamp                                        
MAC000066 2014-01-01 00:00:00                   0.180386   
          2014-01-01 00:30:00                   0.212784   
          2014-01-01 01:00:00                   0.149360   
          2014-01-01 01:30:00                   0.155214   
          2014-01-01 02:00:00                   0.155184   
...                                                  ...   
MAC005521 2014-01-31 21:30:00                   0.108641   
          2014-01-31 22:00:00                   0.109621   
          2014-01-31 22:30:00                   0.092053   
          2014-01-31 23:00:00                   0.111090   
          2014-01-31 23:30:00                   0.218139   

                               Tuned GFM+Meta+Clustered Part  
LCLid     timestamp                                           
MAC000066 2014-01-01 00:00:00                       0.168535  
          2014-01-01 00:30:00                       0.202988  
          2014-01-01 01:00:00                       0.158501  
          2014-01-01 01:30:00                       0.151669  
          2014-01-01 02:00:00                       0.173461  
...                                                      ...  
MAC005521 2014-01-31 21:30:00                       0.142564  
          2014-01-31 22:00:00                       0.118482  
    

### Loading the GFM Forecast and calculating the aggregate metrics on selected LCLids

In [12]:
try:
    global_ml_fc_df = pd.read_pickle(output/"gfm_predictions_val_df.pkl")
    global_ml_fc_df = global_ml_fc_df.loc[global_ml_fc_df.index.get_level_values(0).isin(sel_lclids)]

    baseline_agg_metrics, baseline_metrics_df = evaluate_forecast(y_pred=global_ml_fc_df["GFM+Meta  (NativeLGBM)"], 
                                                     test_target = global_ml_fc_df["energy_consumption"], train_target = train_df["energy_consumption"], model_name="GFM+Meta  (NativeLGBM)")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Global Forecasting Models-ML.ipynb in Chapter10
    </div>
    """))

Calculating metrics...:   0%|          | 0/32 [00:00<?, ?it/s]

# Missing Value Handling

## Null check

In [13]:
nc = train_df.isnull().sum()
nc[nc>0]

pressure                                            674
energy_consumption_lag_1                             32
energy_consumption_lag_2                             64
energy_consumption_lag_3                             96
energy_consumption_lag_4                            128
energy_consumption_lag_5                            160
energy_consumption_lag_46                          1472
energy_consumption_lag_47                          1504
energy_consumption_lag_48                          1536
energy_consumption_lag_49                          1568
energy_consumption_lag_50                          1600
energy_consumption_lag_334                        10688
energy_consumption_lag_335                        10720
energy_consumption_lag_336                        10752
energy_consumption_lag_337                        10784
energy_consumption_lag_338                        10816
energy_consumption_rolling_3_mean                    96
energy_consumption_rolling_3_std                

In [14]:
missing_value_config = MissingValueConfig(
    bfill_columns=[
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
        "energy_consumption_rolling_3_mean",
        "energy_consumption_rolling_3_std",
        "energy_consumption_rolling_6_mean",
        "energy_consumption_rolling_6_std",
        "energy_consumption_rolling_12_mean",
        "energy_consumption_rolling_12_std",
        "energy_consumption_rolling_48_mean",
        "energy_consumption_rolling_48_std",
        "energy_consumption_48_seasonal_rolling_3_mean",
        "energy_consumption_48_seasonal_rolling_3_std",
        "energy_consumption_336_seasonal_rolling_3_mean",
        "energy_consumption_336_seasonal_rolling_3_std",
        "energy_consumption_ewma__span_2880",
        "energy_consumption_ewma__span_336",
        "energy_consumption_ewma__span_48",
    ],
    ffill_columns=[],
    zero_fill_columns=[],
)

In [15]:
train_df = missing_value_config.impute_missing_values(train_df)
test_df = missing_value_config.impute_missing_values(test_df)

/Users/kedarwaghmode/projects/personal/Modern-Time-Series-Forecasting-with-Python/src/forecasting/ml_forecasting.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[bfill_columns] = df[bfill_columns].fillna(method="bfill")
/Users/kedarwaghmode/projects/personal/Modern-Time-Series-Forecasting-with-Python/src/forecasting/ml_forecasting.py:40: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[ffill_columns] = df[ffill_columns].fillna(method="ffill")
/Users/kedarwaghmode/projects/personal/Modern-Time-Series-Forecasting-with-Python/src/forecasting/ml_forecasting.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[bfill_columns] = df[bfill_columns].fillna(method="bfill")
/Users/kedarwaghmode/projects/personal/Mode

In [16]:
nc = train_df.isnull().sum()
nc[nc>0]

Series([], dtype: int64)

# Running DL Global models

In [17]:
metric_record = [baseline_agg_metrics]

In [18]:
# I needed this fix, you may not. Please run next cell and if it doesnt work, come back and uncomment following line and run it.
# ! pip install git+https://github.com/manujosephv/pytorch_tabular.git@modern_ts_freeze

## PyTorch Tabular

In [19]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import FTTransformerConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig

In [20]:
data_config = DataConfig(
    target=[target], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=[
        "visibility",
        "windBearing",
        "temperature",
        "dewPoint",
        "pressure",
        "apparentTemperature",
        "windSpeed",
        "humidity",
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
        "energy_consumption_rolling_3_mean",
        "energy_consumption_rolling_3_std",
        "energy_consumption_rolling_6_mean",
        "energy_consumption_rolling_6_std",
        "energy_consumption_rolling_12_mean",
        "energy_consumption_rolling_12_std",
        "energy_consumption_rolling_48_mean",
        "energy_consumption_rolling_48_std",
        "energy_consumption_48_seasonal_rolling_3_mean",
        "energy_consumption_48_seasonal_rolling_3_std",
        "energy_consumption_336_seasonal_rolling_3_mean",
        "energy_consumption_336_seasonal_rolling_3_std",
        "energy_consumption_ewma_span_2880",
        "energy_consumption_ewma_span_336",
        "energy_consumption_ewma_span_48",
        "timestamp_Elapsed",
        "timestamp_Month_sin_1",
        "timestamp_Month_sin_2",
        "timestamp_Month_sin_3",
        "timestamp_Month_sin_4",
        "timestamp_Month_sin_5",
        "timestamp_Month_cos_1",
        "timestamp_Month_cos_2",
        "timestamp_Month_cos_3",
        "timestamp_Month_cos_4",
        "timestamp_Month_cos_5",
        "timestamp_Hour_sin_1",
        "timestamp_Hour_sin_2",
        "timestamp_Hour_sin_3",
        "timestamp_Hour_sin_4",
        "timestamp_Hour_sin_5",
        "timestamp_Hour_cos_1",
        "timestamp_Hour_cos_2",
        "timestamp_Hour_cos_3",
        "timestamp_Hour_cos_4",
        "timestamp_Hour_cos_5",
        "timestamp_Minute_sin_1",
        "timestamp_Minute_sin_2",
        "timestamp_Minute_sin_3",
        "timestamp_Minute_sin_4",
        "timestamp_Minute_sin_5",
        "timestamp_Minute_cos_1",
        "timestamp_Minute_cos_2",
        "timestamp_Minute_cos_3",
        "timestamp_Minute_cos_4",
        "timestamp_Minute_cos_5",
        "timestamp_Is_quarter_end",
        "timestamp_Is_quarter_start",
        "timestamp_Is_year_end",
        "timestamp_Is_year_start",
        "timestamp_Is_month_start",
    ],
    categorical_cols=[
        "holidays",
        "precipType",
        "icon",
        "summary",
        "timestamp_Month",
        "timestamp_Quarter",
        "timestamp_WeekDay",
        "timestamp_Dayofweek",
        "timestamp_Dayofyear",
        "timestamp_Hour",
        "timestamp_Minute",
        "stdorToU", 
        "Acorn", 
        "Acorn_grouped", 
        "LCLid"
    ],
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=1000,
    auto_select_gpus=True,
    gpus=-1
)
optimizer_config = OptimizerConfig()

In [21]:
# Making categorical columns as string because PyTorch Tabular currently doenst support categorical dtype
train_df[data_config.categorical_cols] = train_df[data_config.categorical_cols].astype(str)
test_df[data_config.categorical_cols] = test_df[data_config.categorical_cols].astype(str)

### FT Transformer Model

In [22]:
model_config = FTTransformerConfig(
    task="regression",
    num_attn_blocks=3,
    num_heads=4,
    transformer_head_dim=64,
    attn_dropout=0.2,
    ff_dropout=0.1,
    out_ff_layers="32",
    learning_rate = 1e-3,
    metrics=["mean_squared_error"]
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [23]:
# Checkpoint stored model
#tabular_model = TabularModel.load_from_checkpoint("notebooks/Chapter12/ft_transformer_global")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kedarwaghmode/projects/personal/Modern-Time-Series-Forecasting-with-Python/notebooks/Chapter12/ft_transformer_global/config.yml'

In [ ]:
tabular_model.fit(train=train_df)
#Saving because you need not train the model again after restarting your kernel or machine etc. and can use the previous cell to load the saved model
tabular_model.save_model("notebooks/Chapter12/ft_transformer_global")
# Deleting automatically saved checkpoints
shutil.rmtree("saved_models")

Global seed set to 42
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This 

/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will ne

/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/tabular_datamodule.py:219: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.35314224 -0.35314224 -0.35314224 ... -0.34564065 -0.16542938
 -0.12394606]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.fit_transform(
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/tabular_datamodule.py:219: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.19525459 -0.19525459 -0.19525459 ...  0.24341121  0.26160051
  0.13427528]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.fit_transform(
/Users/kedarwaghmode/miniconda3/envs/modern_ts

/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/tabular_datamodule.py:219: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 6.40457387e-04  7.07781181e-01  7.07781181e-01 ... -1.41364099e+00
 -1.22416318e+00 -7.06500266e-01]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.fit_transform(
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/tabular_datamodule.py:219: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 6.16722237e-05  1.00007999e+00  1.00007999e+00 ... -9.99956650e-01
 -1.41417783e+00 -9.99956650e-01]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.fit_transform

/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will ne

/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/categorical_encoders.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will ne

/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_tabular/tabular_datamodule.py:112: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  int(self.train[col].fillna("NA").nunique()) + 1
/Users/kedarwaghmode/miniconda3/envs/modern_ts/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
forecast_df = tabular_model.predict(test_df)
pred_df = pred_df.join(forecast_df[["energy_consumption_prediction"]]).rename(
    columns={"enrgy_consumption_prediction": model_config._model_name}
)
agg_metrics, eval_metrics_df = evaluate_forecast(
    y_pred=forecast_df[f"{target}_prediction"],
    test_target=forecast_df["energy_consumption"],
    train_target=train_df["energy_consumption"],
    model_name=model_config._model_name,
)
metric_record.append(agg_metrics)

# Evaluation of ML Forecast

In [ ]:
from src.utils import ts_utils

In [ ]:
agg_metrics_df = pd.DataFrame(metric_record)
agg_metrics_df.style.format({"MAE": "{:.4f}", 
                          "MSE": "{:.4f}", 
                          "meanMASE": "{:.4f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]).apply(highlight_abs_min, props='color:black;background-color:lightgreen', axis=0, subset=['Forecast Bias'])

In [ ]:
metrics_df = pd.concat([eval_metrics_df,baseline_metrics_df])

In [ ]:
fig = px.histogram(metrics_df, 
                   x="MASE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,2])
# fig.write_image("imgs/chapter_12/ft_mase_dist.png")
fig.show()

In [ ]:
fig = px.histogram(metrics_df, 
                   x="MAE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=100, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MAE", ylabel="Probability Density", title="Distribution of MAE in the dataset")
# fig.write_image("imgs/chapter_12/ft_mae_dist.png")
fig.update_layout(xaxis_range=[0,0.2])
fig.show()

In [ ]:
fig = px.histogram(metrics_df, 
                   x="MSE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MSE", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,0.05])
# fig.write_image("imgs/chapter_12/ft_mse_dist.png")
fig.show()

In [ ]:
fig = px.histogram(metrics_df, 
                   x="Forecast Bias", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=250,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="Forecast Bias", ylabel="Probability Density", title="Distribution of Forecast Bias in the dataset")
fig.update_layout(xaxis_range=[-15,10])
# fig.write_image("imgs/chapter_12/ft_bias_dist.png")
fig.show()

# Saving the Forecasts and Metrics

In [ ]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [ ]:
pred_df.to_pickle(output/"ft_transformer_val_df.pkl")
metrics_df.to_pickle(output/"ft_transformer_metrics_val_df.pkl")
agg_metrics_df.to_pickle(output/"ft_transformer_aggregate_metrics_val.pkl")